In [134]:
from io import StringIO
from scipy import stats
from scipy.interpolate import interp1d
from scipy.signal import hilbert, chirp

import seaborn as sns
import pyaudio
import wave
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

In [204]:
sample_format = pyaudio.paInt16  # 16 bits per sample
CHANNELS = 2
RATE = 44100
COUNT = 4
CHUNK = int(RATE / COUNT) # 1024 int(RATE / 4)
TRESHHOLD = 2000
seconds = 2
filename = "output.wav"

In [136]:
def get_medians_from_frame(frame):
    medians = []
    for f in frame:
        medians.append(np.median(frame))
    return medians

In [137]:
def get_squares(frame):
    new_frames = []
    for f in frame:
        new_frames.append(
            # f ** 2
            abs(f)
        )
    return new_frames

In [138]:
def moving_avg(x, n):
 cumsum = np.cumsum(np.insert(x, 0, 0)) 
 return (cumsum[n:] - cumsum[:-n]) / float(n)

In [227]:
# data = np.concatenate(frames)
def process(frame):
    print(f"frame = {frame}")
    # data = np.concatenate(frames)
    # for i in range(len(frames) - COUNT):
    #     print(np.concatenate(frames[i:i+COUNT]))
    # print(get_medians_from_frame(frame))
    frame = get_squares(frame)
    frame = moving_avg(frame, int(CHUNK / 2))
    print(f"moving_avg = {frame}")
    return frame

In [228]:
def plot(frame):
    fig = px.line(np.array(frame))
    fig.show()

=====

In [229]:
p = pyaudio.PyAudio()

stream = p.open(format=pyaudio.paInt16, channels=CHANNELS, rate=RATE, input=True,  frames_per_buffer=CHUNK)
frames = []
global_frames = []

print("Listening...")
for i in range(0, int(RATE / CHUNK * seconds)):
    sample = stream.read(CHUNK)
    frame = np.fromstring(sample, dtype=np.int16)
    frames.append(frame)
    global_frames.append(frame)
    print(len(frames))
    # if len(frames) % COUNT == 0:
        # print("Cough?")
        # by_one_seconds_frame = np.concatenate(frames)
        # print(ln(by_one_seconds_frame))
    process(frame)
        # plot(frames[i])
        # frames = []
        
stream.stop_stream()
stream.close()
p.terminate()

# save
wf = wave.open(filename, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(sample_format))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

Listening...


/tmp/ipykernel_15030/956533731.py:10: DeprecationWarning:

The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead



1
frame = [26 26 26 ... 23 21 21]
moving_avg = [37.99020319 37.99092888 37.99165457 ... 33.80624093 33.80805515
 33.80986938]
2
frame = [ 9  9 25 ... -1  8  8]
moving_avg = [27.67053701 27.68124093 27.69194485 ... 26.84325109 26.83907837
 26.83490566]
3
frame = [  8   8  12 ... 888 833 833]
moving_avg = [4343.72351234 4344.17978955 4344.63606676 ... 1562.36683599 1561.89386792
 1561.42089985]


/tmp/ipykernel_15030/3790329350.py:6: RuntimeWarning:

overflow encountered in scalar absolute



4
frame = [881 881 995 ... 136 129 129]
moving_avg = [575.02467344 574.8717344  574.71879536 ...  56.92416546  56.94430334
  56.96444122]
5
frame = [116 116 111 ...   0  10  10]
moving_avg = [38.55478955 38.54408563 38.53338171 ... 30.60522496 30.60649492
 30.60776488]
6
frame = [  3   3 -15 ...  35  34  34]
moving_avg = [29.60921626 29.61465893 29.6201016  ... 27.95355588 27.95301161
 27.95246734]
7
frame = [ 35  35  40 ... -53 -57 -57]
moving_avg = [27.98730044 27.98167634 27.97605225 ... 27.75907112 27.76342525
 27.76777939]
8
frame = [-58 -58 -61 ...   7   8   8]
moving_avg = [30.36429608 30.35540639 30.34651669 ... 24.51886792 24.51632801
 24.5137881 ]


In [226]:
global_frames

[array([42, 42, 33, ...,  5, -7, -7], dtype=int16),
 array([ -9,  -9,  -6, ..., 565, 538, 538], dtype=int16),
 array([532, 532, 466, ..., -20, -25, -25], dtype=int16),
 array([-24, -24, -27, ...,  14,  17,  17], dtype=int16),
 array([17, 17, 19, ..., 14, 16, 16], dtype=int16),
 array([14, 14,  6, ..., 45, 51, 51], dtype=int16),
 array([ 37,  37,  34, ..., -28, -24, -24], dtype=int16),
 array([-21, -21, -12, ..., -44, -61, -61], dtype=int16)]

In [182]:
d = [1, 2, 3, 4, 5, 6, 7, 8]
for i in range(0, len(d) - 4):
    print(d[i], d[i + 4])

1 5
2 6
3 7
4 8
